In [57]:
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import load_workbook
import math
import time
import io
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files

# lasep22                  is AUG 31 looking into Sep 2022 DONE 2.45
# 2jdfoct22              is SEPT 30 looking into Oct 2022  DONE 2.8
# looking4JDF_intonovember    is OCT looking into Nov 2022 DONE 3.73
# lookingintoDEC22          is NOV30 looking into Dec 2022 DONE 4.08
# lookingintoJAN23       is Dec 2022 looking into Jan 2023 DONE 4.12
#lookingintoFEB23                    is Jan2023 into feb23 DONE 4.6
# lookatmarfrfr23                         is Feb2023 into mar23 4.65

RFR = 2.45

# Mount Google Drive yeehaw
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
from openpyxl import load_workbook
data_file3 = "/content/drive/My Drive/lasep22.csv"
df3 = pd.read_csv(data_file3)
df3.fillna(0, inplace=True)

In [59]:
# pick rows where days to expiry is closest to 30 for each ticker:
#print(df3.head(8))
df3 = df3[df3['C=Call, P=Put'] == 'C']
df3['Abs_Diff'] = abs(df3['D2EXP'] - 30)
df_sorted = df3.sort_values(by=['Ticker Symbol', 'Abs_Diff'])
def filter_min_abs_diff(group):
    min_abs_diff = group['Abs_Diff'].min()
    return group[group['Abs_Diff'] == min_abs_diff]
result = df_sorted.groupby('Ticker Symbol').apply(filter_min_abs_diff).reset_index(drop=True)
result.drop(columns=['Abs_Diff'], inplace=True)
#print(result.head(25))

<ipython-input-59-f13653fbda41>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Abs_Diff'] = abs(df3['D2EXP'] - 30)


In [60]:
# now pick closest strike to price:
result['Abs_PRC_DIFF'] = abs(result['SPRC'] - result['STRK'])
result_sorted = result.sort_values(by=['Ticker Symbol', 'Abs_PRC_DIFF'])
def filter_min_abs_diff(group):
    min_abs_diff = group['Abs_PRC_DIFF'].min()
    return group[group['Abs_PRC_DIFF'] == min_abs_diff]

result_filtered = result_sorted.groupby('Ticker Symbol').apply(filter_min_abs_diff).reset_index(drop=True)
result_filtered.drop(columns=['Abs_PRC_DIFF'], inplace=True)

#print(result_filtered)

In [61]:
#now calc equiv 30-day vols for ea:
#result.rename(columns={'old_name': 'new_name'}, inplace=True)
def equivalent_volatility(row):
    vol_old = row['Implied Volatility of the Option']
    time_old = row['D2EXP']
    time_new = 30
    vol_new = vol_old * np.sqrt(time_new / time_old)
    return vol_new

result_filtered['vol_30_equiv'] = result_filtered.apply(equivalent_volatility, axis=1)
#print(result_filtered)

In [62]:
from scipy.stats import norm
from scipy.optimize import fsolve

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log((S / K) + np.finfo(float).eps) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def implied_volatility(row, r):
    S = row['SPRC']  # Current stock price
    K = row['STRK']  # Strike price
    T = row['D2EXP'] / 365  # Time to expiration in years
    market_price = row['OPRC']  # Market price of the option
    def f(sigma):
        return black_scholes_call(S, K, T, r, sigma) - market_price
    iv = fsolve(f, 0.4)  # Starting guess for volatility is 0.2
    return iv[0]

# Applying implied_volatility function to each row of the DataFrame   # r for nov30 1mo
result_filtered['BS_IV'] = result_filtered.apply(implied_volatility, args=(RFR/100,), axis=1)
#print(result_filtered)

<ipython-input-62-65b1786de7c0>:5: RuntimeWarning: invalid value encountered in log
  d1 = (np.log((S / K) + np.finfo(float).eps) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [63]:
result_filtered_grouped = result_filtered.groupby('Ticker Symbol').mean().reset_index()
#print(result_filtered_grouped)

<ipython-input-63-0d0d4a16545d>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result_filtered_grouped = result_filtered.groupby('Ticker Symbol').mean().reset_index()


In [64]:
result_filtered_grouped = result_filtered_grouped[['Ticker Symbol', 'BS_IV']]
result_filtered_grouped.to_csv('/content/drive/My Drive/ivol2_sep22.csv', index=False)